In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pymongo
client = pymongo.MongoClient()
db = client.metrics

In [3]:
import numpy as np
from collections import defaultdict
from joblib import Parallel, delayed
import dill

from Metrics import rpf1
from load_data import load_process_essays
from wordtagginghelper import merge_dictionaries

#from gensim.models import Word2Vec
from window_based_tagger_config import get_config
from IdGenerator import IdGenerator as idGen
from results_procesor import ResultsProcessor, __MICRO_F1__
from Rpfa import micro_rpfa
from collections import defaultdict

import Settings
import logging
import datetime

In [4]:
import pickle
from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings
from Settings import Settings

CV_FOLDS = 5
DEV_SPLIT = 0.1

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
training_pickled = settings.data_directory + "CoralBleaching/Thesis_Dataset/training.pl"
models_folder = root_folder + "Models/Bi-LSTM/"

config = get_config(training_folder)
processor = ResultsProcessor()

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [5]:
with open(training_pickled, "rb+") as f:
    tagged_essays = pickle.load(f)
len(tagged_essays)

902

In [6]:
import datetime, logging
print("Started at: " + str(datetime.datetime.now()))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger()

Started at: 2017-06-25 13:35:23.391716


In [7]:
CAUSER = "Causer"
RESULT = "Result"
EXPLICIT = "explicit"
CAUSER_EXPLICIT = "Causer_Explicit"
EXPLICIT_RESULT = "Explicit_Result"
CAUSER_EXPLICIT_RESULT = "Causer_Explicit_Result"
CAUSER_RESULT = "Causer_Result"

In [8]:
tag_freq = defaultdict(int)
unique_words = set()
for essay in tagged_essays:
    for sentence in essay.sentences:
        for word, tags in sentence:
            unique_words.add(word)
            for tag in tags:
                tag_freq[tag] += 1

EMPTY_TAG = "Empty"
regular_tags = list((t for t in tag_freq.keys() if ( "->" in t) and not "Anaphor" in t and not "other" in t and not "rhetorical" in t))
#regular_tags = list((t for t in tag_freq.keys() if t[0].isdigit()))

vtags = set(regular_tags)
#vtags.add(EMPTY_TAG)
cr_tags = vtags

len(unique_words)

1641

In [9]:
[k for k in sorted(tag_freq.keys())]

['1',
 '11',
 '12',
 '13',
 '14',
 '2',
 '3',
 '4',
 '5',
 '50',
 '5b',
 '6',
 '7',
 'Anaphor',
 'COMPILED',
 'Causer',
 'Causer:1',
 'Causer:1->Result:11',
 'Causer:1->Result:13',
 'Causer:1->Result:14',
 'Causer:1->Result:2',
 'Causer:1->Result:3',
 'Causer:1->Result:4',
 'Causer:1->Result:5',
 'Causer:1->Result:50',
 'Causer:1->Result:6',
 'Causer:1->Result:7',
 'Causer:1->Result:Anaphor',
 'Causer:1->Result:other',
 'Causer:1->Result:rhetorical',
 'Causer:11',
 'Causer:11->Result:11',
 'Causer:11->Result:12',
 'Causer:11->Result:13',
 'Causer:11->Result:14',
 'Causer:11->Result:3',
 'Causer:11->Result:4',
 'Causer:11->Result:50',
 'Causer:11->Result:6',
 'Causer:11->Result:Anaphor',
 'Causer:11->Result:rhetorical',
 'Causer:12',
 'Causer:12->Result:11',
 'Causer:12->Result:13',
 'Causer:12->Result:14',
 'Causer:12->Result:50',
 'Causer:12->Result:5b',
 'Causer:12->Result:7',
 'Causer:13',
 'Causer:13->Result:11',
 'Causer:13->Result:12',
 'Causer:13->Result:14',
 'Causer:13->Result

In [10]:
[k for k in sorted(tag_freq.keys()) if "->" not in k]

['1',
 '11',
 '12',
 '13',
 '14',
 '2',
 '3',
 '4',
 '5',
 '50',
 '5b',
 '6',
 '7',
 'Anaphor',
 'COMPILED',
 'Causer',
 'Causer:1',
 'Causer:11',
 'Causer:12',
 'Causer:13',
 'Causer:14',
 'Causer:2',
 'Causer:3',
 'Causer:4',
 'Causer:5',
 'Causer:50',
 'Causer:5b',
 'Causer:6',
 'Causer:7',
 'Causer:Anaphor',
 'Causer:other',
 'Causer:rhetorical',
 'Result',
 'Result:1',
 'Result:11',
 'Result:12',
 'Result:13',
 'Result:14',
 'Result:2',
 'Result:3',
 'Result:4',
 'Result:5',
 'Result:50',
 'Result:5b',
 'Result:6',
 'Result:7',
 'Result:Anaphor',
 'Result:other',
 'Result:rhetorical',
 'explicit',
 'other',
 'rhetorical']

In [11]:
sorted(vtags)

['Causer:1->Result:11',
 'Causer:1->Result:13',
 'Causer:1->Result:14',
 'Causer:1->Result:2',
 'Causer:1->Result:3',
 'Causer:1->Result:4',
 'Causer:1->Result:5',
 'Causer:1->Result:50',
 'Causer:1->Result:6',
 'Causer:1->Result:7',
 'Causer:11->Result:11',
 'Causer:11->Result:12',
 'Causer:11->Result:13',
 'Causer:11->Result:14',
 'Causer:11->Result:3',
 'Causer:11->Result:4',
 'Causer:11->Result:50',
 'Causer:11->Result:6',
 'Causer:12->Result:11',
 'Causer:12->Result:13',
 'Causer:12->Result:14',
 'Causer:12->Result:50',
 'Causer:12->Result:5b',
 'Causer:12->Result:7',
 'Causer:13->Result:11',
 'Causer:13->Result:12',
 'Causer:13->Result:14',
 'Causer:13->Result:4',
 'Causer:13->Result:5',
 'Causer:13->Result:50',
 'Causer:13->Result:6',
 'Causer:13->Result:7',
 'Causer:14->Result:50',
 'Causer:14->Result:6',
 'Causer:14->Result:7',
 'Causer:2->Result:1',
 'Causer:2->Result:3',
 'Causer:2->Result:50',
 'Causer:2->Result:6',
 'Causer:3->Result:1',
 'Causer:3->Result:13',
 'Causer:3-

In [12]:
START = "<start>"
END   = "<end>"

def get_training_data(tessays):
    # outputs
    xs = []
    ys = []
    ys_bytag = defaultdict(list)
    ys_bytag_sent = defaultdict(list)
    seq_lens = []

    # cut texts after this number of words (among top max_features most common words)
    for essay in tessays:
        for sentence in essay.sentences:
            row = []
            y_found = False
            y_seq = []
            unique_tags = set()
            for word, tags in [(START, set())] + sentence + [(END, set())]:
                id = generator.get_id(word) #starts at 0, but 0 used to pad sequences
                row.append(id)                
                # remove unwanted tags
                tags = vtags.intersection(tags)
                unique_tags.update(tags)
                # retain all tags for evaluation (not just most common)
                # SKIP the START and END tags
                if word != START and word != END:
                    for t in (vtags - set([EMPTY_TAG])):
                        if t in tags:
                            ys_bytag[t].append(1)
                        else:
                            ys_bytag[t].append(0)

                # encode ys with most common tag only
                if len(tags) > 1:
                    most_common = max(tags, key=lambda t: tag_freq[t])
                    tags = set([most_common])
                if len(tags) == 0:
                    tags.add(EMPTY_TAG)

                one_hot = []
                for t in vtags:
                    if t in tags:
                        one_hot.append(1)
                    else:
                        one_hot.append(0)
                y_seq.append(one_hot)
            
            for tag in vtags:
                if tag in unique_tags:
                    ys_bytag_sent[tag].append(1)
                else:
                    ys_bytag_sent[tag].append(0)
                
            seq_lens.append(len(row)-2)
            ys.append(y_seq)
            xs.append(row)
    
    xs = np.asarray(xs)
    ys = np.asarray(ys)
    assert xs.shape[0] == ys.shape[0], "Sequences should have the same number of rows"
    assert xs.shape[1] == ys.shape[1] == maxlen, "Sequences should have the same lengths"
    return xs, ys, ys_bytag, ys_bytag_sent, seq_lens

### Do CRE Tags Overlap? Yes......

In [91]:
cre = set([CAUSER, EXPLICIT, RESULT])
cnt, total_wds = 0,0
for eix, essay in enumerate(tagged_essays):
    for six, sentence in enumerate(essay.sentences):
        prev_itags = set()
        for wix, (word, tags) in enumerate(sentence):
            total_wds += 1
            itags = cre.intersection(tags)
            if len(itags) > 1:
                new_ones = []
                for t in itags:
                    if t not in prev_itags:
                        new_ones.append(t)
                if len(new_ones) > 1:  
                    print((essay.name, eix,six,wix, str(new_ones), len(itags), ",".join(tags)))
                    cnt+=1
            prev_itags = itags

('EBA1415_BGJD_2_CB_ES-05736.ann', 12, 9, 8, "['Result', 'explicit']", 2, '14,explicit,Causer:4->Result:14,Result,Result:14')
('EBA1415_BGJD_2_CB_ES-05751.ann', 17, 5, 7, "['Result', 'explicit']", 2, '14,explicit,Causer:4->Result:14,Result,Result:14')
('EBA1415_BGJD_2_CB_ES-05756.ann', 21, 1, 8, "['Causer', 'Result']", 2, 'Result:7,7,Result,Causer:7,Causer:7->Result:50,Causer,Causer:3->Result:7')
('EBA1415_BLHT_5_CB_ES-05202.ann', 23, 3, 6, "['Result', 'explicit']", 3, 'Causer:11->Result:12,explicit,Result:12,Result,Causer:12->Result:13,Causer:12,12,Causer')
('EBA1415_BLHT_5_CB_ES-05202.ann', 23, 3, 19, "['Causer', 'Result']", 2, 'Result:13,13,Causer:13,Result,Causer:13->Result:50,Causer:12->Result:13,Causer')
('EBA1415_BLHT_5_CB_ES-05202.ann', 23, 8, 23, "['Causer', 'Result']", 2, 'Causer:7->Result:5b,Causer:5b->Result:50,Result,5b,Causer,Causer:5b,Result:5b')
('EBA1415_BLHT_5_CB_ES-05204.ann', 24, 13, 8, "['Causer', 'Result']", 2, 'Result:50,Causer:50->Result:50,50,Result,Causer:50,C

In [92]:
cnt, total_wds

(559, 137166)

In [94]:
cre = set([CAUSER, EXPLICIT, RESULT])
cnt, total_wds = 0,0
for eix, essay in enumerate(tagged_essays):
    for six, sentence in enumerate(essay.sentences):
        prev_itags = set()
        for wix, (word, tags) in enumerate(sentence):
            total_wds += 1            
            for t in tags:
                if "->" in t:
                    itags = cre.intersection(tags)
                    if len(itags) <=0:
                        print((essay.name, eix,six,wix, len(itags), ",".join(tags)))
            prev_itags = itags

('EBA1415_AEKD_4_CB_ES-05572.ann', 1, 1, 8, 0, 'Causer:5->Result:50')
('EBA1415_AEKD_4_CB_ES-05572.ann', 1, 1, 10, 0, 'Causer:5->Result:50')
('EBA1415_AEKD_4_CB_ES-05902.ann', 2, 0, 21, 0, 'Causer:7->Result:50')
('EBA1415_AEKD_4_CB_ES-05902.ann', 2, 0, 23, 0, 'Causer:7->Result:50')
('EBA1415_AEKD_4_CB_ES-05902.ann', 2, 5, 5, 0, 'Causer:3->Result:4')
('EBA1415_AEKD_4_CB_ES-05902.ann', 2, 5, 6, 0, 'Causer:3->Result:4')
('EBA1415_AEKD_4_CB_ES-05902.ann', 2, 5, 7, 0, 'Causer:3->Result:4')
('EBA1415_AEKD_4_CB_ES-05902.ann', 2, 6, 7, 0, 'Causer:7->Result:50')
('EBA1415_AEKD_5_CB_ES-05582.ann', 4, 1, 7, 0, 'Causer:7->Result:50')
('EBA1415_BGJD_1_CB_ES-05727.ann', 7, 0, 7, 0, 'Causer:11->Result:50,Causer:1->Result:50,Causer:13->Result:50')
('EBA1415_BGJD_1_CB_ES-05727.ann', 7, 0, 7, 0, 'Causer:11->Result:50,Causer:1->Result:50,Causer:13->Result:50')
('EBA1415_BGJD_1_CB_ES-05727.ann', 7, 0, 7, 0, 'Causer:11->Result:50,Causer:1->Result:50,Causer:13->Result:50')
('EBA1415_BGJD_1_CB_ES-05727.ann',

('EBA1415_ERSK_7_CB_ES-06269.ann', 110, 9, 10, 0, 'Causer:6->Result:7')
('EBA1415_ERSK_7_CB_ES-06269.ann', 110, 9, 11, 0, 'Causer:6->Result:7')
('EBA1415_ERSK_7_CB_ES-06269.ann', 110, 9, 12, 0, 'Causer:6->Result:7')
('EBA1415_ERSK_7_CB_ES-06269.ann', 110, 9, 13, 0, 'Causer:6->Result:7')
('EBA1415_ERSK_7_CB_ES-06269.ann', 110, 9, 14, 0, 'Causer:6->Result:7')
('EBA1415_ERSK_7_CB_ES-06269.ann', 110, 11, 6, 0, 'Causer:1->Result:3')
('EBA1415_ERSK_7_CB_ES-06272.ann', 112, 0, 9, 0, 'Causer:7->Result:50')
('EBA1415_HNJD_4_CB_ES-05809.ann', 113, 0, 11, 0, 'Causer:1->Result:50,Causer:13->Result:50,Causer:7->Result:50')
('EBA1415_HNJD_4_CB_ES-05809.ann', 113, 0, 11, 0, 'Causer:1->Result:50,Causer:13->Result:50,Causer:7->Result:50')
('EBA1415_HNJD_4_CB_ES-05809.ann', 113, 0, 11, 0, 'Causer:1->Result:50,Causer:13->Result:50,Causer:7->Result:50')
('EBA1415_HNJD_4_CB_ES-05809.ann', 113, 0, 12, 0, 'Causer:1->Result:50,Causer:13->Result:50,Causer:7->Result:50')
('EBA1415_HNJD_4_CB_ES-05809.ann', 113, 

('EBA1415_KYNS_4_CB_ES-05404.ann', 216, 8, 7, 0, 'Causer:1->Result:3')
('EBA1415_KYNS_4_CB_ES-05404.ann', 216, 8, 8, 0, 'Causer:1->Result:3')
('EBA1415_KYNS_4_CB_ES-05404.ann', 216, 19, 4, 0, 'Causer:3->Result:50')
('EBA1415_KYNS_4_CB_ES-05404.ann', 216, 19, 8, 0, 'Causer:3->Result:50,Causer:7->Result:50')
('EBA1415_KYNS_4_CB_ES-05404.ann', 216, 19, 8, 0, 'Causer:3->Result:50,Causer:7->Result:50')
('EBA1415_KYNS_4_CB_ES-05404.ann', 216, 19, 9, 0, 'Causer:3->Result:50,Causer:7->Result:50')
('EBA1415_KYNS_4_CB_ES-05404.ann', 216, 19, 9, 0, 'Causer:3->Result:50,Causer:7->Result:50')
('EBA1415_KYNS_4_CB_ES-05404.ann', 216, 19, 10, 0, 'Causer:3->Result:50,Causer:7->Result:50')
('EBA1415_KYNS_4_CB_ES-05404.ann', 216, 19, 10, 0, 'Causer:3->Result:50,Causer:7->Result:50')
('EBA1415_KYNS_4_CB_ES-05404.ann', 216, 20, 12, 0, 'Causer:11->Result:14')
('EBA1415_KYNS_4_CB_ES-05405.ann', 217, 0, 2, 0, 'Causer:rhetorical->Result:50')
('EBA1415_KYNS_4_CB_ES-05405.ann', 217, 0, 5, 0, 'Causer:rhetorical->

('EBA1415_SEAL_34_CB_ES-04792.ann', 399, 2, 12, 0, 'Causer:7->Result:Anaphor')
('EBA1415_SEAL_34_CB_ES-04792.ann', 399, 2, 13, 0, 'Causer:7->Result:Anaphor')
('EBA1415_SEAL_34_CB_ES-04792.ann', 399, 2, 14, 0, 'Causer:7->Result:Anaphor')
('EBA1415_SEAL_34_CB_ES-04792.ann', 399, 2, 16, 0, 'Causer:7->Result:Anaphor')
('EBA1415_SEAL_34_CB_ES-04792.ann', 399, 2, 17, 0, 'Causer:7->Result:Anaphor')
('EBA1415_SEAL_34_CB_ES-04792.ann', 399, 2, 18, 0, 'Causer:7->Result:Anaphor')
('EBA1415_SEAL_34_CB_ES-04792.ann', 399, 3, 7, 0, 'Causer:6->Result:7')
('EBA1415_SEAL_34_CB_ES-04792.ann', 399, 3, 8, 0, 'Causer:6->Result:7')
('EBA1415_SEAL_34_CB_ES-04796.ann', 400, 4, 10, 0, 'Causer:4->Result:50')
('EBA1415_SEAL_34_CB_ES-04796.ann', 400, 4, 11, 0, 'Causer:4->Result:50')
('EBA1415_SEAL_34_CB_ES-04796.ann', 400, 4, 12, 0, 'Causer:4->Result:50')
('EBA1415_SEAL_34_CB_ES-04796.ann', 400, 7, 5, 0, 'Causer:3->Result:14')
('EBA1415_SEAL_34_CB_ES-04796.ann', 400, 8, 5, 0, 'Causer:4->Result:50')
('EBA1415_SEAL

('EBA1415_TTCM_2_CB_ES-06141.ann', 692, 0, 13, 0, 'Causer:5->Result:50,Causer:1->Result:50')
('EBA1415_TTCM_2_CB_ES-06141.ann', 692, 5, 1, 0, 'Causer:Anaphor->Result:50')
('EBA1415_TTCM_2_CB_ES-06141.ann', 692, 5, 2, 0, 'Causer:Anaphor->Result:50')
('EBA1415_TTCM_2_CB_ES-06141.ann', 692, 5, 3, 0, 'Causer:Anaphor->Result:50')
('EBA1415_TTCM_2_CB_ES-06141.ann', 692, 5, 4, 0, 'Causer:Anaphor->Result:50')
('EBA1415_TTCM_2_CB_ES-06141.ann', 692, 5, 5, 0, 'Causer:Anaphor->Result:50')
('EBA1415_TTCM_2_CB_ES-06142.ann', 693, 3, 8, 0, 'Causer:3->Result:5')
('EBA1415_TTCM_2_CB_ES-06143.ann', 694, 3, 12, 0, 'Causer:3->Result:5')
('EBA1415_TTCM_2_CB_ES-06143.ann', 694, 8, 3, 0, 'Causer:14->Result:50')
('EBA1415_TTCM_2_CB_ES-06143.ann', 694, 8, 4, 0, 'Causer:14->Result:50')
('EBA1415_TTCM_2_CB_ES-06143.ann', 694, 8, 5, 0, 'Causer:14->Result:50')
('EBA1415_TTCM_2_CB_ES-06143.ann', 694, 8, 6, 0, 'Causer:14->Result:50')
('EBA1415_TTCM_2_CB_ES-06143.ann', 694, 8, 15, 0, 'Result:50,50,Causer:7->Result:5

#### Any C->E Sequence Lack all 3 Sub-tags?

In [97]:
from collections import defaultdict

cre = set([CAUSER, EXPLICIT, RESULT])
cnt, total_wds = 0,0
for eix, essay in enumerate(tagged_essays):
    for six, sentence in enumerate(essay.sentences):
        cr_words = defaultdict(list)
        cr_tags = defaultdict(list)
        for wix, (word, tags) in enumerate(sentence):
            total_wds += 1            
            for t in tags:
                if "->" in t:
                    cr_words[t].append(word)
                    itags = cre.intersection(tags)
                    cr_tags[t].append(itags)
        
        for tag, lst in cr_tags.items():
            utags = set()
            for tgs in lst:
                utags.update(tgs)
            if len(utags) < 3:
                print((essay.name, eix, six, tag, len(utags), ",".join(utags)))

('EBA1415_AEKD_4_CB_ES-05902.ann', 2, 5, 'Causer:3->Result:4', 2, 'Causer,Result')
('EBA1415_BGJD_1_CB_ES-05727.ann', 7, 0, 'Causer:13->Result:50', 2, 'Causer,Result')
('EBA1415_BGJD_1_CB_ES-05727.ann', 7, 0, 'Causer:11->Result:50', 2, 'Causer,Result')
('EBA1415_BGJD_1_CB_ES-05727.ann', 7, 0, 'Causer:1->Result:50', 2, 'Causer,Result')
('EBA1415_BGJD_2_CB_ES-05740.ann', 14, 0, 'Causer:1->Result:50', 2, 'Causer,Result')
('EBA1415_BGJD_2_CB_ES-05740.ann', 14, 1, 'Causer:1->Result:3', 2, 'Causer,Result')
('EBA1415_BGJD_2_CB_ES-05751.ann', 17, 6, 'Causer:4->Result:5', 2, 'Causer,Result')
('EBA1415_BGJD_2_CB_ES-05752.ann', 18, 1, 'Causer:3->Result:50', 2, 'Causer,Result')
('EBA1415_BGJD_2_CB_ES-05752.ann', 18, 3, 'Causer:3->Result:50', 2, 'Causer,Result')
('EBA1415_BGJD_2_CB_ES-05756.ann', 21, 1, 'Causer:3->Result:7', 2, 'Causer,Result')
('EBA1415_BLHT_5_CB_ES-05202.ann', 23, 4, 'Causer:3->Result:50', 2, 'Causer,Result')
('EBA1415_BLHT_5_CB_ES-05202.ann', 23, 8, 'Causer:7->Result:5b', 2, 'Ca

('EBA1415_SEAL_78_CB_ES-04804.ann', 412, 3, 'Causer:1->Result:50', 2, 'Causer,Result')
('EBA1415_SEAL_78_CB_ES-04804.ann', 412, 4, 'Causer:1->Result:50', 2, 'Causer,Result')
('EBA1415_SEAL_78_CB_ES-04804.ann', 412, 8, 'Causer:2->Result:3', 2, 'Causer,Result')
('EBA1415_SEAL_78_CB_ES-04806.ann', 413, 10, 'Causer:4->Result:5', 2, 'Causer,Result')
('EBA1415_SEAL_78_CB_ES-04806.ann', 413, 12, 'Causer:3->Result:4', 2, 'Causer,Result')
('EBA1415_SEAL_78_CB_ES-04806.ann', 413, 14, 'Causer:1->Result:50', 2, 'Causer,Result')
('EBA1415_SEAL_78_CB_ES-04806.ann', 413, 14, 'Causer:3->Result:50', 2, 'Causer,Result')
('EBA1415_SEAL_78_CB_ES-04808.ann', 414, 0, 'Causer:3->Result:50', 2, 'Causer,Result')
('EBA1415_SEAL_78_CB_ES-04808.ann', 414, 5, 'Causer:3->Result:4', 2, 'Causer,Result')
('EBA1415_SEKL_1_CB-04816.ann', 416, 0, 'Causer:3->Result:4', 2, 'Causer,Result')
('EBA1415_SEKL_1_CB-04816.ann', 416, 3, 'Causer:3->Result:50', 2, 'Causer,Result')
('EBA1415_SEKL_1_CB-04816.ann', 416, 3, 'Causer:1->R

### Are there Words with Multiple CR Tags? Yes also...

In [22]:
from collections import defaultdict

cnt, total_wds, total_sents, sents_with_cr = 0,0,0,0
sents_wds = defaultdict(list)
for eix, essay in enumerate(tagged_essays):
    for six, sentence in enumerate(essay.sentences):
        total_sents += 1
        has_cr_tag = False
        for wix, (word, tags) in enumerate(sentence):
            total_wds += 1
            crel_tags = set()
            for t in tags:
                if "->" in t:
                    has_cr_tag = True
                    crel_tags.add(t)
            if len(crel_tags) > 1:            
                sents_wds[(eix, six)].append(wix)
                #print((essay.name, eix, six, wix, tags,  crel_tags))
        if has_cr_tag:
            sents_with_cr += 1

In [23]:
len(sents_wds), total_sents, sents_with_cr

(597, 8292, 2611)

In [39]:
RESULT

'Result'

In [88]:
#for (eix, six), wds in list(sents_wds.items())[:10]:
for (eix, six), wds in list(sents_wds.items())[:10]:
    eix = 25
    six = 0
    
    essay = tagged_essays[eix]
    print(essay.name)
    print()
    sentence = essay.sentences[six]
    for wd, tags in sentence:
        csr_tags = ",".join(sorted([t for t in tags if "->" not in t and "Causer:" in t]))
        res_tags = ",".join(sorted([t for t in tags if "->" not in t and "Result:" in t]))
        cr_tags = ",".join(sorted([t.replace("Causer:","").replace("Result:","") for t in tags if "->" in t]))
        reg_tags = ",".join(sorted([t for t in tags if "->" not in t and (t[0].isdigit() or t == EXPLICIT)]))
        print(wd.ljust(15), reg_tags.ljust(15), cr_tags.ljust(30), csr_tags.ljust(20), res_tags)
    print("*" * 100)

EBA1415_BLHT_5_CB_ES-05205.ann

there                                                                               
are                                                                                 
many                                                                                
factors                                                                             
that                                                                                
can                                                                                 
cause                                                                               
coral           50              1->50,5->50,6->50                                   Result:50
bleaching       50              1->50,5->50,6->50                                   Result:50
.                                                                                   
****************************************************************************************************

### Look For Non Projective Relations

In [84]:
for (eix, six), wds in list(sents_wds.items())[:10]:
    essay = tagged_essays[eix]
    sentence = essay.sentences[six]
    un_tags = []
    seen = set()
    c_rels = set()
    for wd, tags in sentence:
        #print(reg_tags, tags)
        for t in tags:
            if t not in seen:
                seen.add(t)
                if "->" not in t and (t[0].isdigit() or "Anaphor" == t or "rhetorical" == t):
                    un_tags.append(t)
                elif "->" in t:
                    c_rels.add(t)
    positions = []
    for cr in c_rels:
        left, right = cr.replace("Causer:","").replace("Result:","").split("->")
        if left not in un_tags or right not in un_tags:
            print("lr:",left,right)
            print("ix:",eix,six)
            print()
            continue
        l_ix = un_tags.index(left)
        r_ix = un_tags.index(right)
        if l_ix > r_ix:
            # swap so l_ix < r_ix
            r_ix, l_ix = l_ix, r_ix
        positions.append((l_ix, r_ix))
    
    non_proj = set()
    for (l,r) in positions:
        for (l1,r1) in positions:
            if l == l1 and r == r1:
                continue
            if (l1 > l and l1 < r) or (r1 > l and r1 < r):
                non_proj.add((eix, six))
    
non_proj

lr: 5 50
ix: 25 0

lr: 6 50
ix: 25 0

lr: 1 50
ix: 25 0



set()

In [91]:
tagged_essays[25].sentences[1]

[('some',
  {'Causer:1->Result:50', 'Causer:5->Result:50', 'Causer:6->Result:50'}),
 ('happen',
  {'Causer:1->Result:50', 'Causer:5->Result:50', 'Causer:6->Result:50'}),
 ('naturally',
  {'Causer:1->Result:50', 'Causer:5->Result:50', 'Causer:6->Result:50'}),
 ('like',
  {'Causer:1->Result:50',
   'Causer:5->Result:50',
   'Causer:6->Result:50',
   'explicit'}),
 ('shifting',
  {'1',
   'Causer',
   'Causer:1',
   'Causer:1->Result:50',
   'Causer:5->Result:50',
   'Causer:6->Result:50'}),
 ('trade',
  {'1',
   'Causer',
   'Causer:1',
   'Causer:1->Result:50',
   'Causer:5->Result:50',
   'Causer:6->Result:50'}),
 ('winds',
  {'1',
   'Causer',
   'Causer:1',
   'Causer:1->Result:50',
   'Causer:5->Result:50',
   'Causer:6->Result:50'}),
 ('and', {'Causer:5->Result:50', 'Causer:6->Result:50'}),
 ('photosynthesis',
  {'5', 'Causer', 'Causer:5', 'Causer:5->Result:50', 'Causer:6->Result:50'}),
 (',', set()),
 ('but', {'Causer:6->Result:50'}),
 ('others', {'Causer:6->Result:50'}),
 ('are',

In [108]:
del cr_tags

## Feature Extraction

In [13]:
WINDOW_SIZE = 7

config["window_size"] = WINDOW_SIZE
offset = int((config["window_size"] - 1) / 2)
offset

3

In [14]:
from featureextractionfunctions \
    import fact_extract_positional_word_features_stemmed, fact_extract_ngram_features_stemmed,\
    fact_extract_bow_ngram_features, extract_dependency_relation, extract_brown_cluster

unigram_window_stemmed  = fact_extract_positional_word_features_stemmed(offset)
biigram_window_stemmed  = fact_extract_ngram_features_stemmed(offset, 2)
triigram_window_stemmed = fact_extract_ngram_features_stemmed(offset, 3)
unigram_bow_window      = fact_extract_bow_ngram_features(offset, 1)

#TODO - add in full feature set, but keep simple for now
#optimal CB feature set
extractors = [
    unigram_window_stemmed,
    biigram_window_stemmed,
    #triigram_window_stemmed,
    #unigram_bow_window,
    #extract_dependency_relation,
    #extract_brown_cluster
]

In [15]:
from featureextractortransformer import FeatureExtractorTransformer

feature_extractor = FeatureExtractorTransformer(extractors)
essay_feats = feature_extractor.transform(tagged_essays)
len(tagged_essays), len(essay_feats)

(902, 902)

In [16]:
l = list(range(0))
l

[]

In [17]:
class SearnModel(object):
    Shift = "Shift"
    Reduce = "Reduce"
    Skip = "Skip"
    
    def __init__(self, feat_extractor, tags, cr_tags, base_learner_fact, beta):
        # init checks
        assert CAUSER in tags, "%s must be in tags" % CAUSER
        assert RESULT in tags, "%s must be in tags" % RESULT
        assert EXPLICIT in tags, "%s must be in tags" % EXPLICIT

        self.feat_extractor = feature_extractor    # feature extractor (for use later)
        self.base_learner_fact = base_learner_fact # SKlearn classifier
        self.tags = set(tags)                      # tags for basic tagging
        self.cr_tags = set(cr_tags)                # causal relation tags
            
        self.actions = set([SearnModel.Shift, SearnModel.Reduce, SearnModel.Skip])
        self.epoch = -1
        self.beta = beta
        self.stack = []
        self.tagging_models = {}
        self.parser_model = None
    
    def get_conditional_feats(self, current_word, prev_words, history):
        feats = {}
        if len(history) > 0:
            #TODO add feats for tags not predicted and actions not taken
            prev_tags = history[-1]
            for t in prev_tags:
                feats["tag-1:" + t] = 1
                feats["tag-1:" + t + " wd:"  + current_word] = 1
                feats["tag-1:" + t + " wd-1:"+ prev_words[-1]] = 1
                
            if len(history) > 1:
                prev_prev_tags = history[-2]
                for t in prev_prev_tags:
                    # tag bigrams
                    for prev_tag in prev_tags:
                        feats["tag-1:" + prev_tag + " tag-2:" + t] = 1
                    feats["tag-2:" + t] = 1
                    feats["tag-2:" + t + " wd:"   + current_word] = 1
                    feats["tag-2:" + t + " wd-2:" + prev_words[-2]] = 1
        
        for i in range(4):
            offset = -(i+1)
            if len(self.stack) < abs(offset):
                break
            s1_key = "s" + str(offset) + ":" + self.stack[offset]
            feats[s1_key] = 1z
            # Stack Bigram feats
            if len(self.stack) >= abs(offset) + 1:
                next_offset = offset-1
                s2_key = "s" + str(next_offset) + ":" + self.stack[next_offset]
                feats[s1_key + " " + s2_key] = 1
        return feats
    
    def __oracle_tagger__(self, word, prev_words, feats, prediction_history, ground_truth, ground_truth_history):
        
        # TODO For model to make predictions (in future)
        # get all tagger predictions for previous 2 tags
        cond_feats = self.get_conditional_feats(word, previous_words, prediction_history, ground_truth_history)
        cond_feats.update(feats)

        # filter to desired tags
        ground_truth_tags = ground_truth.intersection(self.tags)
        return ground_truth_tags
    
    def __oracle_parser__(self, feats, pred_tags, prediction_history, ground_truth, ground_truth_history):

        # TODO For model to make predictions (in future)
        cond_feats = self.get_conditional_feats(word, previous_words, prediction_history + [pred_tags], ground_truth_history)
        cond_feats.update(feats)

        if len(ground_truth_history) > 0:
            prev_ground_truth_tags = ground_truth_history[-1]
        else:
            prev_ground_truth_tags = set()
        #Decide Actions
        decisions = set()
        pushed = 0
        #TODO what if multiple causer, explicit or results tags assigned to one word?
        if CAUSER in ground_truth and CAUSER not in prev_ground_truth_tags:
            decisions.add(SearnModel.Shift)
            self.stack.append(CAUSER)
            pushed += 1
        elif EXPLICIT in ground_truth and EXPLICIT not in prev_ground_truth_tags:
            decisions.add(SearnModel.Shift)
            self.stack.append(EXPLICIT)
            pushed +=1
        elif RESULT in ground_truth and RESULT not in prev_ground_truth_tags:
            decisions.add(SearnModel.Shift)
            self.stack.append(RESULT)
            pushed +=1
        else:
            pred_tags.add(SearnModel.Skip)
            
        # TODO - for reduce model
        dec_feats = self.get_conditional_feats(word, previous_words, \
                        prediction_history + [pred_tags.union(decisions)], ground_truth_history)
        dec_feats.update(feats)
        # Shift, Reduce or Skip?
        # How to handle end of input (do we try and reduce what's remaining) ?
        #TODO - prevent predicting concept code causes same concept code
        #TODO - ensure that causer code is less than result code? (there are some exceptions though to this)
        causal_relns = ground_truth.intersection(self.cr_tags)        
        if len(causal_relns) > 0 and len(self.stack) >=2:
            top_two = list(sorted(self.stack[-2:]))
            
            #TODO - are there ever > 1 causal relation assigned to the same word?
            for i in range(2):
                push = None
                if top_two == [CAUSER, EXPLICIT]:
                    push = CAUSER_EXPLICIT
                elif top_two == [EXPLICIT, RESULT]:
                    push = EXPLICIT_RESULT
                elif top_two in [[CAUSER_EXPLICIT, EXPLICIT_RESULT],
                                 [CAUSER,          EXPLICIT_RESULT],
                                 [CAUSER_EXPLICIT, RESULT],
                                 [CAUSER,          RESULT]
                                ]:
                    push = CAUSER_EXPLICIT_RESULT
                if push:
                    decisions.add(SearnModel.Reduce)
                    self.stack = self.stack[:-2] + [push]
                
                #TODO - determine if can have multiple Causal reln's on same word
                # if so, continue, otherwise remove loop
                # also - need to keep track of causal_relations - do we pop and remember?
                if self.stack[-1] in []
                    # reduce into CER
                else:
                    break
                
            for t in cr_tags:
                #TODO: implement parsing actions
                pass
        pass

    def __oracle__(self, word, prev_words, feats, prediction_history, ground_truth, ground_truth_history):
        pred_tags = self.__oracle_tagger__(word, prev_words, cond_feats, prediction_history, ground_truth, ground_truth_history)
        parse_decisions = self.__oracle_parser__(feats, pred_tags, ground_truth, ground_truth_history)
        return pred_tags, parse_decisions
            
    def train(self, essay_feats, epochs):
        #essay_feats = self.feat_extractor.transform(tagged_essays)        
        for i in range(0, epochs):
            self.epoch +=1
            tagging_models = {}
            examples_with_loss = []
            
            for essay_ix, essay in enumerate(essay_feats):
                for sent_ix, taggged_sentence in enumerate(essay.sentences):
                    
                    prediction_history = []
                    ground_truth_history = []
                    previous_words = []
                    self.stack = []
                    for i, (wd) in enumerate(taggged_sentence):
                        
                        # Don't mutate the feat dictionary
                        tagger_feats = dict(wd.features.items())
                        
                        # add more in depth features for this tag
                        ground_truth = wd.tags
                        
                        word, prev_words, feats, prediction_history, ground_truth, ground_truth_history):
        
                        # get all tagger predictions for previous 2 tags
                        #cond_feats = self.get_conditional_feats(wd.word, previous_words, prediction_history, ground_truth_history)

                        oracle_tags, oracle_parse_decisions = \
                            self.__oracle__(wd.word, previous_words, tagger_feats, prediction_history, ground_truth, ground_truth_history)                
                    
                        previous_words.append(wd.word)
                        prediction_history.append(oracle_predictions)
                        ground_truth_history.append(ground_truth)
                        
                        #TODO - compute loss, add examples with associated losses

SyntaxError: invalid syntax (<ipython-input-17-8bb8a3d84a39>, line 49)

In [105]:
a = [1,2]
b = [3,4]

e = [1,2]

e in [b,b]

False